# WebbPSF Tutorial

***

## Introduction
**WebbPSF** is a realistic point spread function (PSF) simulator developed by the Space Telescope Science Institute that, despite the name, supports both Roman and the James Webb Space Telescope. For Roman's Wide Field Imager (WFI), it harnesses the NASA Goddard Space Flight Center's latest optical models (including field-dependent aberrations across the sensor chip assembly) to create these simulations. There is also an experimental module for Roman's Coronagraph Instrument.

This notebook serves as an introduction to simulating Roman WFI PSFs and computing standard PSF properties like full width at half maximum and encircled energy. Before you begin, be sure that you have not only installed WebbPSF but also downloaded its required input data files.

## Imports

Besides `webbpsf`, the `matplotlib`-related imports will help visualize simulated PSFs and their derivatives.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib
import webbpsf

We also edit `matplotlib` configuration parameters to make figures larger by default and prevent any smoothing of pixel edges.

In [ ]:
matplotlib.rcParams['figure.figsize'] = (16, 7)
matplotlib.rcParams['image.interpolation'] = 'nearest'

***

## Examples

### Quick PSF generation

#### Default parameters

After selecting the proper instrument (the Roman WFI), generating a PSF can be as short as a single line.

In [ ]:
wfi = webbpsf.roman.WFI()
default_psf = wfi.calc_psf(display=True)

The resulting object contains a list of two FITS extensions, both containing idealized PSF images. `OVERSAMP` splits the detector's original number of pixels by the oversampling factor specified upon creation of the PSFs (by default, 4) while `DET_SAMP` retains the original pixel count.

_Note: Additional extensions that apply distortion to the idealized PSFs are in development for the `WFI` model._

In [ ]:
default_psf.info()

The package includes a helper function for visualizing each extension of an existing PSF.

In [ ]:
webbpsf.display_psf(default_psf, ext='OVERSAMP')

In [ ]:
webbpsf.display_psf(default_psf, ext='DET_SAMP')

The PSF object itself contains functionality for saving its contents to disk for later use.

In [ ]:
default_psf.writeto('default_PSF.fits', overwrite=True)

#### Custom parameters

You can simulate custom PSFs by modifying instrument class attributes like the detector, filter, and others. (Find all available attributes in the API documentation for the [Roman `WFI` class](https://webbpsf.readthedocs.io/en/latest/api/webbpsf.WFI.html#webbpsf.WFI) and [its parent `SpaceTelescopeInstrument` class](https://webbpsf.readthedocs.io/en/latest/api/webbpsf.SpaceTelescopeInstrument.html#webbpsf.SpaceTelescopeInstrument).)

In [ ]:
wfi.filter_list

In [ ]:
wfi.filter = 'GRISM1'
wfi.detector = 'SCA14'
wfi.detector_position = (1024, 1024)
 
custom_psf = wfi.calc_psf(display=True)

### PSF profile and encircled energy

Once you've simulated a PSF, WebbPSF offers a method that measures its profile, full width and half maximum (FWHM), and encircled energy.

In [ ]:
src = webbpsf.specFromSpectralType('G0V', catalog='phoenix')
poly_psf = wfi.calc_psf(source=src, nlambda=10, display=False)

In [ ]:
plt.figure(figsize=(8, 6))
webbpsf.display_profiles(poly_psf)

### Bulk PSF generation

To generate several individual PSFs in a grid, WebbPSF implements the `GriddedPSFModel` class from `photutils`. (Learn more about this use case [in the WebbPSF documentation](https://webbpsf.readthedocs.io/en/latest/psf_grids.html).)

As a toy example, we generate a 3x3 grid of PSFs for a single WFI detector below.

In [ ]:
wfi.detector = "SCA07"
wfi.filter = "F158"
wfi_grid = wfi.psf_grid(num_psfs=9, all_detectors=False)

Use another WebbPSF helper function to visualize the grid.

The top figure displays PSFs in each detector position specified by the grid. The bottom figure subtracts the mean PSF from each corresponding image from the top to show the effect of detector position on image quality.

In [ ]:
webbpsf.gridded_library.display_psf_grid(wfi_grid)

With a PSF grid in hand, you can use a package like `photutils` to estimate PSFs at any given detector position by interpolation without the need to calculate each PSF individually. This is a major time-saver in source-heavy scientific use cases.

## Aditional Resources

- The Roman User Documentation's ["WebbPSF Overview"](https://roman-docs.stsci.edu/simulation-tools-handbook-home/webbpsf-for-roman/overview-of-webbpsf) page.
- The latest version of the [WebbPSF documentation](https://webbpsf.readthedocs.io/en/stable/index.html) on ReadTheDocs, which includes a specific page on the package's [Roman instrument model](https://webbpsf.readthedocs.io/en/stable/roman.html).
- The [Roman Help Desk](https://roman-docs.stsci.edu/roman-help-desk-at-stsci), an official outlet for user questions about WebbPSF.

## About this notebook

**Author:** Justin Otor, Staff Scientist II.  
**Updated In:** 2024-05

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 